In [1]:
import sqlite3
import pandas as pd
import numpy as np

### Pre-processing

In [2]:
with sqlite3.connect('../data/main.db') as conn:
    df = pd.read_sql('select * from measurements where resolver is null ', con=conn)

In [3]:
df.head()

,msm_id,domain,resolver,is_tls,read
0,22202814,google.com,None,0,1
1,22202849,youtube.com,None,0,1
2,22202942,facebook.com,None,0,1
3,22203337,baidu.com,None,0,1
4,22203711,wikipedia.org,None,0,1


In [4]:
df.dtypes

msm_id       int64
domain      object
resolver    object
is_tls       int64
read         int64
dtype: object

In [5]:
len(df['msm_id'].unique())

2796

In [6]:
# read all local resolvers, i.e., ones where resolver is null
with sqlite3.connect('../data/main.db') as conn:
    df = pd.read_sql('select * \
                      from results \
                      where msm_id in (select distinct msm_id \
                                       from measurements \
                                       where resolver is null)', con=conn)

In [7]:
df.dtypes

msm_id           int64
domain          object
resolver        object
prb_id           int64
rt             float64
size           float64
src_addr        object
dst_addr        object
timestamp        int64
is_tls           int64
ttl            float64
return_code     object
err              int64
err_msg         object
dtype: object

In [8]:
len(df['msm_id'].unique())

2796

In [9]:
len(df)

23723680

In [10]:
df

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg
0,22202814,google.com,None,10006,28.197,44.0,192.168.0.106,8.8.8.8,1562122806,0,37.0,NOERROR,0,None
1,22202814,google.com,None,10007,15.952,44.0,192.168.7.24,75.75.75.75,1562122813,0,248.0,NOERROR,0,None
2,22202814,google.com,None,10007,31.730,44.0,192.168.7.24,75.75.76.76,1562122813,0,215.0,NOERROR,0,None
3,22202814,google.com,None,10019,29.383,44.0,192.168.31.41,192.168.31.1,1562122806,0,131.0,NOERROR,0,None
4,22202814,google.com,None,10023,29.930,315.0,172.31.253.253,172.31.0.42,1562122807,0,300.0,NOERROR,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23723675,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True}
23723676,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True}
23723677,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True}
23723678,22299007,sberbank.ru,None,27827,NaN,NaN,None,None,1563456195,1,NaN,None,1,{u'TU_BAD_ADDR': True}


In [11]:
import pyasn

In [12]:
asndb = pyasn.pyasn('../metadata/ipasn-07-2019.dat')

In [13]:
def lookup_asn(ip):
    if ip is None:
        return None
    else:
        asn, prefix = asndb.lookup(ip)
        return asn

In [14]:
df['asn'] = df['dst_addr'].map(lookup_asn)

In [15]:
import json

In [16]:
with open('../metadata/as_names.json') as f:
    as_names_dict = json.load(f)
as_names_df = pd.DataFrame.from_dict(as_names_dict, orient='index').reset_index(level=0)
as_names_df.rename(columns={'index' : 'asn', 0: 'as_holder'}, inplace=True)
as_names_df['asn'] = as_names_df['asn'].astype(int)

In [17]:
df = df.merge(as_names_df, how='left', on='asn')

In [18]:
df

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
0,22202814,google.com,None,10006,28.197,44.0,192.168.0.106,8.8.8.8,1562122806,0,37.0,NOERROR,0,None,15169.0,"GOOGLE, US"
1,22202814,google.com,None,10007,15.952,44.0,192.168.7.24,75.75.75.75,1562122813,0,248.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
2,22202814,google.com,None,10007,31.730,44.0,192.168.7.24,75.75.76.76,1562122813,0,215.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
3,22202814,google.com,None,10019,29.383,44.0,192.168.31.41,192.168.31.1,1562122806,0,131.0,NOERROR,0,None,NaN,NaN
4,22202814,google.com,None,10023,29.930,315.0,172.31.253.253,172.31.0.42,1562122807,0,300.0,NOERROR,0,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23723675,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723676,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723677,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723678,22299007,sberbank.ru,None,27827,NaN,NaN,None,None,1563456195,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [19]:
pub_resolvers = pd.read_csv('../metadata/resolvers-extended.csv')

In [20]:
pub_resolvers_list = pub_resolvers['ip'].unique()

In [21]:
with sqlite3.connect('../data/main.db') as conn:
    resolvers = pd.read_sql('select * \
                             from resolvers', con=conn)

In [22]:
resolvers

,id,name,ipv4,ipv4_2,tls
0,1,CleanBrowsing,185.228.168.168,185.228.169.168,True
1,2,Cloudflare 1.1.1.1,1.1.1.1,1.0.0.1,True
2,3,Comodo Secure DNS,8.26.56.26,8.20.247.20,False
3,4,CZ.NIC ODVR,217.31.204.130,193.29.206.206,False
4,5,Oracle + Dyn,216.146.35.35,216.146.36.36,False
5,6,DNS.WATCH,84.200.69.80,84.200.70.40,False
6,7,Google Public DNS,8.8.8.8,8.8.4.4,True
7,8,Neustar UltraRecursive,156.154.70.1,156.154.71.1,False
8,9,OpenDNS,208.67.222.222,208.67.220.220,False
9,10,OpenNIC,185.121.177.177,169.239.202.202,False


In [23]:
resolver_ips = np.concatenate([resolvers['ipv4'].unique(), resolvers['ipv4_2'].unique(), pub_resolvers_list])

In [24]:
resolver_ips

array(['185.228.168.168', '1.1.1.1', '8.26.56.26', '217.31.204.130',
       '216.146.35.35', '84.200.69.80', '8.8.8.8', '156.154.70.1',
       '208.67.222.222', '185.121.177.177', '9.9.9.9', '195.46.39.39',
       '91.239.100.100', '64.6.64.6', '77.88.8.1', '185.228.169.168',
       '1.0.0.1', '8.20.247.20', '193.29.206.206', '216.146.36.36',
       '84.200.70.40', '8.8.4.4', '156.154.71.1', '208.67.220.220',
       '169.239.202.202', '149.112.112.112', '195.46.39.40',
       '89.233.43.71', '64.6.65.6', '77.88.8.8', '185.228.168.168',
       '1.1.1.1', '8.8.8.8', '208.67.222.123', '185.121.177.177',
       '9.9.9.9', '64.6.64.6', '77.88.8.8', '156.154.70.1',
       '2a0d:2a00:1::1', '2606:4700:4700::1111', '2001:4860:4860::8888',
       '2620:0:ccc::2', '2a05:dfc7:5::5353', '2620:fe::9',
       '2620:74:1b::1:1', '2a02:6b8::feed:ff', '2610:a1:1018::1',
       '1.0.0.1', '2606:4700:4700::1001', '2001:4860:4860::8844',
       '8.8.4.4', '208.67.220.220', '208.67.222.220', '208.67.222.22

In [25]:
true_local = df[~df['dst_addr'].isin(resolver_ips)]

In [26]:
true_local

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1,22202814,google.com,None,10007,15.952,44.0,192.168.7.24,75.75.75.75,1562122813,0,248.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
2,22202814,google.com,None,10007,31.730,44.0,192.168.7.24,75.75.76.76,1562122813,0,215.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
3,22202814,google.com,None,10019,29.383,44.0,192.168.31.41,192.168.31.1,1562122806,0,131.0,NOERROR,0,None,NaN,NaN
4,22202814,google.com,None,10023,29.930,315.0,172.31.253.253,172.31.0.42,1562122807,0,300.0,NOERROR,0,None,NaN,NaN
5,22202814,google.com,None,10023,60.962,292.0,172.31.253.253,172.31.0.43,1562122807,0,300.0,NOERROR,0,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23723675,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723676,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723677,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723678,22299007,sberbank.ru,None,27827,NaN,NaN,None,None,1563456195,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [27]:
true_local['as_holder'].value_counts()

SIDN, NL                                                                        454883
LIBERTYGLOBAL Liberty Global (formerly UPC Broadband Holding, aka AORTA), AT     82334
COMCAST-7922, US                                                                 70868
BELWUE BelWue-Koordination, EU                                                   66262
LEVEL3, US                                                                       47992
                                                                                 ...  
KABEL-TV-LAMPERT, AT                                                               159
MOLDCELL_AS str. Belgrad 3, MD                                                     129
CHINA169-BACKBONE CHINA UNICOM China169 Backbone, CN                               104
TELE2, EU                                                                           11
SNAP-NZ-AS Snap Internet Limited, NZ                                                 6
Name: as_holder, Length: 1039, dtype: int64

In [28]:
resolvers['name'].values

array(['CleanBrowsing', 'Cloudflare 1.1.1.1', 'Comodo Secure DNS',
       'CZ.NIC ODVR', 'Oracle + Dyn', 'DNS.WATCH', 'Google Public DNS',
       'Neustar UltraRecursive', 'OpenDNS', 'OpenNIC', 'Quad9', 'SafeDNS',
       'UncensoredDNS', 'VeriSign Public DNS', 'Yandex.DNS'], dtype=object)

In [29]:
resolver_list = ['CleanBrowsing', 'Cloudflare', 'Comodo',
       'CZ.NIC', 'Oracle', 'Dyn', 'DNS.WATCH', 'Google',
       'Neustar', 'OpenDNS', 'OpenNIC', 'Quad9', 'SafeDNS',
       'UncensoredDNS', 'VeriSign', 'Yandex']

In [30]:
true_local = true_local[~(true_local['as_holder'
                                    ].str.lower().str.contains('|'.join([x.lower() for x in resolver_list])
                                                              )
                         ).fillna(False)
                       ]

In [31]:
true_local

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1,22202814,google.com,None,10007,15.952,44.0,192.168.7.24,75.75.75.75,1562122813,0,248.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
2,22202814,google.com,None,10007,31.730,44.0,192.168.7.24,75.75.76.76,1562122813,0,215.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
3,22202814,google.com,None,10019,29.383,44.0,192.168.31.41,192.168.31.1,1562122806,0,131.0,NOERROR,0,None,NaN,NaN
4,22202814,google.com,None,10023,29.930,315.0,172.31.253.253,172.31.0.42,1562122807,0,300.0,NOERROR,0,None,NaN,NaN
5,22202814,google.com,None,10023,60.962,292.0,172.31.253.253,172.31.0.43,1562122807,0,300.0,NOERROR,0,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23723675,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723676,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723677,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723678,22299007,sberbank.ru,None,27827,NaN,NaN,None,None,1563456195,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [32]:
home_probes_only = pd.read_csv('../metadata/home-probes-merged.csv')

In [33]:
home_probes_only

,prb_id
0,10006
1,10007
2,10080
3,10087
4,10092
...,...
3226,35719
3227,35723
3228,35724
3229,35735


In [34]:
faulty_probes = pd.read_csv('../metadata/faulty-probes.csv')

In [35]:
faulty_probes

,prb_id
0,10048
1,10058
2,11608
3,11743
4,12087
5,12321
6,12880
7,15355
8,15762
9,16759


In [36]:
# filter by home probes and faulty probes
true_local = true_local[true_local['prb_id'].isin(home_probes_only['prb_id'].unique())]
true_local = true_local[~(true_local['prb_id'].isin(faulty_probes['prb_id'].unique()))]

In [37]:
true_local

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1,22202814,google.com,None,10007,15.952,44.0,192.168.7.24,75.75.75.75,1562122813,0,248.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
2,22202814,google.com,None,10007,31.730,44.0,192.168.7.24,75.75.76.76,1562122813,0,215.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
24,22202814,google.com,None,10080,5.981,44.0,192.168.0.50,192.168.0.1,1562122810,0,15.0,NOERROR,0,None,NaN,NaN
27,22202814,google.com,None,10087,28.309,44.0,192.168.218.6,193.25.222.254,1562122806,0,300.0,NOERROR,0,None,196890.0,"COMPLEX-AS, PL"
28,22202814,google.com,None,10087,NaN,NaN,None,None,1562122806,0,NaN,None,1,{u'timeout': 5000},NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23723671,22299007,sberbank.ru,None,34979,NaN,NaN,None,None,1563191807,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723674,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723675,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
23723676,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [40]:
# write intermediate results
with sqlite3.connect('../data/loc-res.db') as conn:
    true_local.to_sql('local_resolvers', index=False, if_exists='replace', con=conn)

In [ ]:
# filter success/fails
# split by Do53 and DoT

# percentages etc.

In [41]:
len(true_local['prb_id'].unique())

3062

In [42]:
success = true_local[true_local['err'] == 0]

In [43]:
do53 = success[success['is_tls'] == 0]
dot = success[success['is_tls'] == 1]

In [44]:
success

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1,22202814,google.com,None,10007,15.952,44.0,192.168.7.24,75.75.75.75,1562122813,0,248.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
2,22202814,google.com,None,10007,31.730,44.0,192.168.7.24,75.75.76.76,1562122813,0,215.0,NOERROR,0,None,7922.0,"COMCAST-7922, US"
24,22202814,google.com,None,10080,5.981,44.0,192.168.0.50,192.168.0.1,1562122810,0,15.0,NOERROR,0,None,NaN,NaN
27,22202814,google.com,None,10087,28.309,44.0,192.168.218.6,193.25.222.254,1562122806,0,300.0,NOERROR,0,None,196890.0,"COMPLEX-AS, PL"
33,22202814,google.com,None,10095,19.909,44.0,192.168.1.37,202.169.192.61,1562122822,0,203.0,NOERROR,0,None,18199.0,"LINKTELECOM-NZ-AP Link Telecom (NZ) Limited, NZ"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721431,22299007,sberbank.ru,None,29989,545.418,45.0,10.0.1.124,194.109.6.66,1562697214,1,3596.0,NOERROR,0,None,3265.0,"XS4ALL-NL Amsterdam, NL"
23721845,22299007,sberbank.ru,None,30899,162.648,219.0,192.168.42.3,46.182.19.48,1562697214,1,3598.0,NOERROR,0,None,43847.0,"NBISERV-AS, DE"
23722390,22299007,sberbank.ru,None,32271,254.773,56.0,192.168.0.20,176.103.130.132,1562697214,1,2155.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722391,22299007,sberbank.ru,None,32271,226.664,56.0,192.168.0.20,176.103.130.134,1562697214,1,932.0,NOERROR,0,None,199274.0,"MNGTNET, RU"


In [45]:
len(success['prb_id'].unique())

2718

In [46]:
len(do53['prb_id'].unique())

2718

In [47]:
len(dot['prb_id'].unique())

13

In [48]:
dot

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1626060,22217033,google.com,None,10342,109.980,44.0,192.168.0.166,103.247.37.37,1562166010,1,300.0,NOERROR,0,None,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU"
1627863,22217033,google.com,None,14377,454.226,124.0,192.168.1.238,192.109.42.42,1562166007,1,300.0,NOERROR,0,None,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE"
1628358,22217033,google.com,None,16233,156.254,44.0,80.127.100.88,194.109.6.66,1562166003,1,300.0,NOERROR,0,None,3265.0,"XS4ALL-NL Amsterdam, NL"
1628751,22217033,google.com,None,17525,379.416,44.0,192.168.1.91,80.67.188.188,1562166007,1,266.0,NOERROR,0,None,60197.0,"LDN, FR"
1628776,22217033,google.com,None,17609,67.936,44.0,10.81.0.122,89.32.32.32,1562166009,1,272.0,NOERROR,0,None,1653.0,"SUNET SUNET Swedish University Network, EU"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721431,22299007,sberbank.ru,None,29989,545.418,45.0,10.0.1.124,194.109.6.66,1562697214,1,3596.0,NOERROR,0,None,3265.0,"XS4ALL-NL Amsterdam, NL"
23721845,22299007,sberbank.ru,None,30899,162.648,219.0,192.168.42.3,46.182.19.48,1562697214,1,3598.0,NOERROR,0,None,43847.0,"NBISERV-AS, DE"
23722390,22299007,sberbank.ru,None,32271,254.773,56.0,192.168.0.20,176.103.130.132,1562697214,1,2155.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722391,22299007,sberbank.ru,None,32271,226.664,56.0,192.168.0.20,176.103.130.134,1562697214,1,932.0,NOERROR,0,None,199274.0,"MNGTNET, RU"


In [49]:
dot['as_holder'].value_counts()

XS4ALL-NL Amsterdam, NL                            8372
FIBERDIREKT, SE                                    2790
MNGTNET, RU                                        2772
IN-BERLIN-AS Individual Network Berlin e.V., DE    1396
LDN, FR                                            1396
NEXTDNS, US                                        1396
DNSFILTER-AS-AP DNSFilter, Inc., AU                1394
SUNET SUNET Swedish University Network, EU         1392
NBISERV-AS, DE                                      194
Name: as_holder, dtype: int64

In [50]:
dot['asn'].value_counts()

3265.0      8372
42303.0     2790
199274.0    2772
29670.0     1396
60197.0     1396
34939.0     1396
64089.0     1394
1653.0      1392
43847.0      194
Name: asn, dtype: int64

In [51]:
dot[dot['as_holder'] == 'XS4ALL-NL Amsterdam, NL']['prb_id'].unique()

array([16233, 20324, 22890, 29989, 32430])

In [52]:
dot['prb_id'].unique()  # probes with local DoT support

array([10342, 14377, 16233, 17525, 17609, 18275, 20324, 22890, 23214,
       29989, 32271, 32430, 30899])

In [53]:
len(dot['prb_id'].unique())

13

In [54]:
with sqlite3.connect('../data/main.db') as conn:
    probes = pd.read_sql('select * from probes', con=conn)

In [55]:
probes[probes['probe_id'].isin(dot['prb_id'].unique())]['country_code'].value_counts()

NL    5
DE    2
US    2
SE    2
FR    1
RO    1
Name: country_code, dtype: int64

In [56]:
import pycountry_convert as pc

In [57]:
do53 = do53.merge(probes, left_on='prb_id', right_on='probe_id', how='left')
do53['continent_code'] = do53['country_code'].map(pc.country_alpha2_to_continent_code)

dot = dot.merge(probes, left_on='prb_id', right_on='probe_id', how='left')
dot['continent_code'] = dot['country_code'].map(pc.country_alpha2_to_continent_code)

In [58]:
cols = ['msm_id', 'timestamp', 'domain',
        'prb_id', 'country_code', 'continent_code',
        'src_addr', 'dst_addr', 'asn', 'as_holder',
        'size', 'return_code', 'rt', 'ttl']

do53 = do53[cols]
dot = dot[cols]

In [59]:
do53.head()

,msm_id,timestamp,domain,prb_id,country_code,continent_code,src_addr,dst_addr,asn,as_holder,size,return_code,rt,ttl
0,22202814,1562122813,google.com,10007,US,NA,192.168.7.24,75.75.75.75,7922.0,"COMCAST-7922, US",44.0,NOERROR,15.952,248.0
1,22202814,1562122813,google.com,10007,US,NA,192.168.7.24,75.75.76.76,7922.0,"COMCAST-7922, US",44.0,NOERROR,31.730,215.0
2,22202814,1562122810,google.com,10080,JP,AS,192.168.0.50,192.168.0.1,NaN,NaN,44.0,NOERROR,5.981,15.0
3,22202814,1562122806,google.com,10087,PL,EU,192.168.218.6,193.25.222.254,196890.0,"COMPLEX-AS, PL",44.0,NOERROR,28.309,300.0
4,22202814,1562122822,google.com,10095,NZ,OC,192.168.1.37,202.169.192.61,18199.0,"LINKTELECOM-NZ-AP Link Telecom (NZ) Limited, NZ",44.0,NOERROR,19.909,203.0


In [60]:
dot.head()

,msm_id,timestamp,domain,prb_id,country_code,continent_code,src_addr,dst_addr,asn,as_holder,size,return_code,rt,ttl
0,22217033,1562166010,google.com,10342,US,NA,192.168.0.166,103.247.37.37,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU",44.0,NOERROR,109.980,300.0
1,22217033,1562166007,google.com,14377,DE,EU,192.168.1.238,192.109.42.42,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",124.0,NOERROR,454.226,300.0
2,22217033,1562166003,google.com,16233,NL,EU,80.127.100.88,194.109.6.66,3265.0,"XS4ALL-NL Amsterdam, NL",44.0,NOERROR,156.254,300.0
3,22217033,1562166007,google.com,17525,FR,EU,192.168.1.91,80.67.188.188,60197.0,"LDN, FR",44.0,NOERROR,379.416,266.0
4,22217033,1562166009,google.com,17609,SE,EU,10.81.0.122,89.32.32.32,1653.0,"SUNET SUNET Swedish University Network, EU",44.0,NOERROR,67.936,272.0


In [61]:
with sqlite3.connect('../data/loc-res.db') as conn:
    do53.to_sql('do53', index=False, if_exists='replace', con=conn)
    dot.to_sql('dot', index=False, if_exists='replace', con=conn)

### Response Times

In [ ]:
# check minimums or 5th percentiles for each probe

In [62]:
do53.groupby('prb_id')['rt'].describe(percentiles=[0.01, 0.05, 0.25, 0.50, 0.75, 0.95])

,count,mean,std,min,1%,5%,25%,50%,75%,95%,max
prb_id,,,,,,,,,,,
10007,2790.0,52.450243,131.381913,13.004,13.61978,14.16090,15.44000,28.6450,32.46900,205.06710,3725.262
10080,1400.0,30.309886,238.171413,0.527,5.22731,5.49085,5.79800,6.0725,7.29250,55.97740,4212.549
10087,1394.0,303.360978,495.707882,0.642,22.89379,23.06465,50.90750,138.1995,319.15675,1222.81840,4142.396
10095,2780.0,375.880815,370.543251,19.563,19.99937,20.42100,146.19325,305.3540,495.21875,1021.82425,3541.096
10098,1399.0,53.129410,181.422768,0.990,8.59976,8.65400,8.79200,9.0150,15.29950,253.42870,2820.484
...,...,...,...,...,...,...,...,...,...,...,...
35699,1395.0,218.802236,272.381655,0.774,1.49576,5.06700,50.93350,105.7460,299.51450,680.10130,2296.916
35701,1373.0,301.384073,306.792726,0.635,39.25044,53.92800,116.80900,219.4740,365.32200,840.60980,3827.293
35719,2789.0,62.387469,84.519012,0.695,13.99376,16.14040,32.22800,41.2580,66.48300,153.35560,1222.729


In [63]:
dot.groupby('prb_id')['rt'].describe(percentiles=[0.01, 0.05, 0.25, 0.50, 0.75, 0.95])

,count,mean,std,min,1%,5%,25%,50%,75%,95%,max
prb_id,,,,,,,,,,,
10342,1394.0,466.646738,443.820037,100.472,101.75092,104.00170,175.64825,329.9215,577.92875,1266.60770,3369.645
14377,1396.0,495.311597,416.800667,174.964,178.36215,191.90675,228.42625,353.6550,581.77575,1300.43475,3426.581
16233,1395.0,289.077553,338.439141,146.218,147.31278,148.06850,149.42350,151.5850,247.11950,1005.39340,3451.883
17525,1396.0,246.936539,243.051226,119.917,121.13650,122.72300,128.31725,155.8540,275.01450,654.16450,3714.372
17609,1392.0,68.801388,9.978848,65.224,65.93783,66.42740,67.27250,67.8375,68.44200,70.48145,339.317
18275,1396.0,400.994604,67.205002,377.606,381.02055,383.77400,386.65950,388.3810,391.08450,444.91950,1533.130
20324,2792.0,171.520614,109.424318,153.819,155.58410,156.55865,158.16250,159.8100,161.34950,168.49345,3213.378
22890,1394.0,428.401622,488.408945,145.189,145.79295,146.84850,152.71625,201.6465,456.16075,1677.20185,4012.359
23214,2790.0,375.950487,335.749498,104.557,107.06323,117.80155,155.03475,245.4330,468.65000,1039.88250,2279.510


In [64]:
do53_5pct = do53.groupby(['prb_id', 'country_code', 'continent_code'])['rt'].quantile(0.05).reset_index()

In [65]:
do53_5pct

,prb_id,country_code,continent_code,rt
0,10007,US,NA,14.16090
1,10080,JP,AS,5.49085
2,10087,PL,EU,23.06465
3,10095,NZ,OC,20.42100
4,10098,GB,EU,8.65400
...,...,...,...,...
2713,35699,SE,EU,5.06700
2714,35701,US,NA,53.92800
2715,35719,NL,EU,16.14040
2716,35723,KZ,AS,12.99740


In [66]:
do53_5pct.groupby('continent_code')['rt'].median()

continent_code
AF     7.619900
AS     7.101175
EU     8.290950
NA    12.413750
OC    10.251450
SA     9.141200
Name: rt, dtype: float64

In [67]:
dot_5pct = dot.groupby(['prb_id', 'country_code', 'continent_code'])['rt'].quantile(0.05).reset_index()

In [68]:
dot_5pct

,prb_id,country_code,continent_code,rt
0,10342,US,NA,104.00170
1,14377,DE,EU,191.90675
2,16233,NL,EU,148.06850
3,17525,FR,EU,122.72300
4,17609,SE,EU,66.42740
5,18275,US,NA,383.77400
6,20324,NL,EU,156.55865
7,22890,NL,EU,146.84850
8,23214,SE,EU,117.80155
9,29989,NL,EU,145.90230


In [69]:
dot_5pct.groupby('continent_code')['rt'].median()

continent_code
EU    148.06850
NA    243.88785
Name: rt, dtype: float64

In [70]:
do53_median = do53.groupby(['prb_id', 'country_code', 'continent_code'])['rt'].median().reset_index()
dot_median = dot.groupby(['prb_id', 'country_code', 'continent_code'])['rt'].median().reset_index()

In [71]:
with sqlite3.connect('../data/loc-res.db') as conn:
    do53_5pct.to_sql('do53_5pct', index=False, if_exists='replace', con=conn)
    dot_5pct.to_sql('dot_5pct', index=False, if_exists='replace', con=conn)
    do53_median.to_sql('do53_median', index=False, if_exists='replace', con=conn)
    dot_median.to_sql('dot_median', index=False, if_exists='replace', con=conn)

In [72]:
do53_5pct

,prb_id,country_code,continent_code,rt
0,10007,US,NA,14.16090
1,10080,JP,AS,5.49085
2,10087,PL,EU,23.06465
3,10095,NZ,OC,20.42100
4,10098,GB,EU,8.65400
...,...,...,...,...
2713,35699,SE,EU,5.06700
2714,35701,US,NA,53.92800
2715,35719,NL,EU,16.14040
2716,35723,KZ,AS,12.99740


In [73]:
dot_5pct

,prb_id,country_code,continent_code,rt
0,10342,US,NA,104.00170
1,14377,DE,EU,191.90675
2,16233,NL,EU,148.06850
3,17525,FR,EU,122.72300
4,17609,SE,EU,66.42740
5,18275,US,NA,383.77400
6,20324,NL,EU,156.55865
7,22890,NL,EU,146.84850
8,23214,SE,EU,117.80155
9,29989,NL,EU,145.90230


In [74]:
delta_5pct = do53_5pct.merge(dot_5pct.drop(columns=['country_code', 'continent_code']),
                             how='outer', on=['prb_id'], suffixes=('_do53', '_dot'))

In [75]:
delta_5pct = delta_5pct.dropna()

In [76]:
delta_5pct['rt_delta'] = delta_5pct['rt_do53'] - delta_5pct['rt_dot'] 

In [77]:
delta_5pct

,prb_id,country_code,continent_code,rt_do53,rt_dot,rt_delta
31,10342,US,NA,10.6125,104.00170,-93.38920
554,14377,DE,EU,21.3388,191.90675,-170.56795
714,16233,NL,EU,4.2550,148.06850,-143.81350
857,17525,FR,EU,7.7536,122.72300,-114.96940
865,17609,SE,EU,1.0610,66.42740,-65.36640
914,18275,US,NA,78.8898,383.77400,-304.88420
1115,20324,NL,EU,6.8814,156.55865,-149.67725
1360,22890,NL,EU,4.5404,146.84850,-142.30810
1390,23214,SE,EU,1.0560,117.80155,-116.74555
1967,29989,NL,EU,5.0730,145.90230,-140.82930


In [78]:
with sqlite3.connect('../data/loc-res.db') as conn:
    delta_5pct.to_sql('delta_5pct', index=False, if_exists='replace', con=conn)

### Failure Rate

In [79]:
true_local['err_msg'].value_counts()

{u'TU_BAD_ADDR': True}                                             3992263
{u'timeout': 5000}                                                 1590663
{u'TUCONNECT': u'Connection refused'}                               746515
{u'socket': u'connect failed Invalid argument'}                     225771
{u'TUCONNECT': u'No route to host'}                                 126037
{u'TUCONNECT': u'Network is unreachable'}                            48848
{u'socket': u'connect failed Network is unreachable'}                47361
{u'TUCONNECT': u'Permission denied'}                                 25108
{u'nameserver': u'no local resolvers found'}                           251
{u'TUCONNECT': u'Connection reset by peer'}                            167
{u'TUCONNECT': u'Success'}                                               9
{u'idmismatch': u'mismatch id from tcp fd 16'}                           6
{u'TUCONNECT': u'error:140900EF:lib(20):func(144):reason(239)'}          2
{u'idmismatch': u'mismatc

Calculate both success and failure rates (should be "1 - success rate = failure rate", calculate both just to make sure)

In [80]:
len(true_local[true_local['err'] != 0]) / len(true_local)

0.5973019846162586

--> 59.7% FAILURE rate in general

In [81]:
len(true_local[(true_local['err'] != 0) & (true_local['is_tls'] == 0)]) / len(true_local[true_local['is_tls'] == 0])

0.11200975467627144

--> 11.2% FAILURE rate for Do53

In [82]:
len(true_local[(true_local['err'] != 0) & (true_local['is_tls'] == 1)]) / len(true_local[true_local['is_tls'] == 1])

0.9966227224742759

--> 99.7% FAILURE rate for DoT

In [83]:
do53_failures = true_local[(true_local['err'] == 1) & (true_local['is_tls'] == 0)]

In [84]:
dot_failures = true_local[(true_local['err'] == 1) & (true_local['is_tls'] == 1)]

In [85]:
do53_failures

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
28,22202814,google.com,None,10087,NaN,NaN,None,None,1562122806,0,NaN,None,1,{u'timeout': 5000},NaN,NaN
30,22202814,google.com,None,10092,NaN,NaN,None,None,1562122808,0,NaN,None,1,{u'socket': u'connect failed Network is unreac...,NaN,NaN
31,22202814,google.com,None,10092,NaN,NaN,None,None,1562122808,0,NaN,None,1,{u'timeout': 5000},NaN,NaN
36,22202814,google.com,None,10099,NaN,NaN,None,None,1562122811,0,NaN,None,1,{u'socket': u'connect failed Invalid argument'},NaN,NaN
73,22202814,google.com,None,10238,NaN,NaN,None,None,1562122810,0,NaN,None,1,{u'timeout': 5000},NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21964703,22295900,sberbank.ru,None,33085,NaN,NaN,None,None,1562675124,0,NaN,None,1,{u'timeout': 5000},NaN,NaN
21964743,22295900,sberbank.ru,None,35149,NaN,NaN,None,None,1562675122,0,NaN,None,1,{u'socket': u'connect failed Network is unreac...,NaN,NaN
21964759,22295900,sberbank.ru,None,35723,NaN,NaN,None,None,1562675123,0,NaN,None,1,{u'timeout': 5000},NaN,NaN
21964770,22295900,sberbank.ru,None,27180,NaN,NaN,None,None,1562847998,0,NaN,None,1,{u'socket': u'connect failed Invalid argument'},NaN,NaN


In [86]:
probes['continent_code'] = probes[probes['country_code'] != '']['country_code'].map(pc.country_alpha2_to_continent_code)

In [87]:
probes = probes[['probe_id', 'country_code', 'continent_code']].rename(columns={'probe_id' : 'prb_id'})

In [88]:
probes

,prb_id,country_code,continent_code
0,10003,NL,EU
1,10006,GB,EU
2,10007,US,NA
3,10019,FR,EU
4,10023,DE,EU
...,...,...,...
5224,35733,IR,AS
5225,35734,JO,AS
5226,35735,,NaN
5227,35742,ZA,AF


In [89]:
do53_failures = do53_failures.merge(probes, on='prb_id', how='left')

In [90]:
dot_failures = dot_failures.merge(probes, on='prb_id', how='left')

In [91]:
do53_failures['err_msg'].value_counts()

{u'timeout': 5000}                                       302719
{u'socket': u'connect failed Invalid argument'}          225771
{u'socket': u'connect failed Network is unreachable'}     47361
{u'nameserver': u'no local resolvers found'}                 28
Name: err_msg, dtype: int64

In [92]:
(_/len(dot_failures)).round(3)

{u'timeout': 5000}                                       0.049
{u'socket': u'connect failed Invalid argument'}          0.036
{u'socket': u'connect failed Network is unreachable'}    0.008
{u'nameserver': u'no local resolvers found'}             0.000
Name: err_msg, dtype: float64

In [93]:
dot_failures['err_msg'].value_counts()

{u'TU_BAD_ADDR': True}                                             3992263
{u'timeout': 5000}                                                 1287944
{u'TUCONNECT': u'Connection refused'}                               746515
{u'TUCONNECT': u'No route to host'}                                 126037
{u'TUCONNECT': u'Network is unreachable'}                            48848
{u'TUCONNECT': u'Permission denied'}                                 25108
{u'nameserver': u'no local resolvers found'}                           223
{u'TUCONNECT': u'Connection reset by peer'}                            167
{u'TUCONNECT': u'Success'}                                               9
{u'idmismatch': u'mismatch id from tcp fd 16'}                           6
{u'TUCONNECT': u'error:140900EF:lib(20):func(144):reason(239)'}          2
{u'idmismatch': u'mismatch id from tcp fd 108'}                          1
{u'idmismatch': u'mismatch id from tcp fd 114'}                          1
{u'idmismatch': u'mismatc

In [94]:
(_/len(dot_failures)).round(3)

{u'TU_BAD_ADDR': True}                                             0.641
{u'timeout': 5000}                                                 0.207
{u'TUCONNECT': u'Connection refused'}                              0.120
{u'TUCONNECT': u'No route to host'}                                0.020
{u'TUCONNECT': u'Network is unreachable'}                          0.008
{u'TUCONNECT': u'Permission denied'}                               0.004
{u'nameserver': u'no local resolvers found'}                       0.000
{u'TUCONNECT': u'Connection reset by peer'}                        0.000
{u'TUCONNECT': u'Success'}                                         0.000
{u'idmismatch': u'mismatch id from tcp fd 16'}                     0.000
{u'TUCONNECT': u'error:140900EF:lib(20):func(144):reason(239)'}    0.000
{u'idmismatch': u'mismatch id from tcp fd 108'}                    0.000
{u'idmismatch': u'mismatch id from tcp fd 114'}                    0.000
{u'idmismatch': u'mismatch id from tcp fd 124'}    

In [95]:
do53_failures.groupby('continent_code').size()

continent_code
AF     12678
AS     35700
EU    410439
NA     87843
OC     17979
SA     11231
dtype: int64

In [96]:
dot_failures.groupby('continent_code').size()

continent_code
AF      80363
AS     489222
EU    4145715
NA    1170529
OC     202851
SA     138434
dtype: int64

In [97]:
do53.groupby('continent_code').size()

continent_code
AF      80319
AS     424178
EU    2952532
NA     838596
OC     155377
SA     114448
dtype: int64

In [98]:
dot.groupby('continent_code').size()

continent_code
EU    18312
NA     2790
dtype: int64

In [99]:
do53_failures

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder,country_code,continent_code
0,22202814,google.com,None,10087,NaN,NaN,None,None,1562122806,0,NaN,None,1,{u'timeout': 5000},NaN,NaN,PL,EU
1,22202814,google.com,None,10092,NaN,NaN,None,None,1562122808,0,NaN,None,1,{u'socket': u'connect failed Network is unreac...,NaN,NaN,BG,EU
2,22202814,google.com,None,10092,NaN,NaN,None,None,1562122808,0,NaN,None,1,{u'timeout': 5000},NaN,NaN,BG,EU
3,22202814,google.com,None,10099,NaN,NaN,None,None,1562122811,0,NaN,None,1,{u'socket': u'connect failed Invalid argument'},NaN,NaN,US,NA
4,22202814,google.com,None,10238,NaN,NaN,None,None,1562122810,0,NaN,None,1,{u'timeout': 5000},NaN,NaN,RU,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575874,22295900,sberbank.ru,None,33085,NaN,NaN,None,None,1562675124,0,NaN,None,1,{u'timeout': 5000},NaN,NaN,CN,AS
575875,22295900,sberbank.ru,None,35149,NaN,NaN,None,None,1562675122,0,NaN,None,1,{u'socket': u'connect failed Network is unreac...,NaN,NaN,AU,OC
575876,22295900,sberbank.ru,None,35723,NaN,NaN,None,None,1562675123,0,NaN,None,1,{u'timeout': 5000},NaN,NaN,KZ,AS
575877,22295900,sberbank.ru,None,27180,NaN,NaN,None,None,1562847998,0,NaN,None,1,{u'socket': u'connect failed Invalid argument'},NaN,NaN,RU,EU


In [100]:
dot_failures

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder,country_code,continent_code
0,22217033,google.com,None,10007,NaN,NaN,None,None,1562166016,1,NaN,None,1,{u'timeout': 5000},NaN,NaN,US,NA
1,22217033,google.com,None,10007,NaN,NaN,None,None,1562166016,1,NaN,None,1,{u'timeout': 5000},NaN,NaN,US,NA
2,22217033,google.com,None,10007,NaN,NaN,None,None,1562166016,1,NaN,None,1,{u'timeout': 5000},NaN,NaN,US,NA
3,22217033,google.com,None,10080,NaN,NaN,None,None,1562166010,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN,JP,AS
4,22217033,google.com,None,10087,NaN,NaN,None,None,1562166007,1,NaN,None,1,{u'TUCONNECT': u'Connection refused'},NaN,NaN,PL,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227121,22299007,sberbank.ru,None,34979,NaN,NaN,None,None,1563191807,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN,IE,EU
6227122,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN,CN,AS
6227123,22299007,sberbank.ru,None,33346,NaN,NaN,None,None,1563192397,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN,CN,AS
6227124,22299007,sberbank.ru,None,34961,NaN,NaN,None,None,1563193184,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN,DE,EU


In [101]:
special_probes = dot['prb_id'].unique()  # probes for which we see AT LEAST ONE successful DoT measurement (=supported)
special_probes

array([10342, 14377, 16233, 17525, 17609, 18275, 20324, 22890, 23214,
       29989, 32271, 32430, 30899])

In [102]:
do53_reg = do53[~do53['prb_id'].isin(special_probes)]
do53_special = do53[do53['prb_id'].isin(special_probes)]
dot_reg = dot[~dot['prb_id'].isin(special_probes)]
dot_special = dot[dot['prb_id'].isin(special_probes)]

In [103]:
do53_reg_failures = do53_failures[~do53_failures['prb_id'].isin(special_probes)]
do53_special_failures = do53_failures[do53_failures['prb_id'].isin(special_probes)]
dot_reg_failures = dot_failures[~dot_failures['prb_id'].isin(special_probes)]
dot_special_failures = dot_failures[dot_failures['prb_id'].isin(special_probes)]

In [104]:
failure_reg_probes = pd.DataFrame([do53_reg_failures.groupby('continent_code').size(),
                                   dot_reg_failures.groupby('continent_code').size(),
                                   do53_reg.groupby('continent_code').size(),
                                   dot_reg.groupby('continent_code').size()])

failure_reg_probes = failure_reg_probes.transpose()
failure_reg_probes.columns = ['do53_failures', 'dot_failures', 'do53_successes', 'dot_successes']
failure_reg_probes['do53_total'] = failure_reg_probes['do53_failures'] + failure_reg_probes['do53_successes']
failure_reg_probes['dot_total'] = failure_reg_probes['dot_failures'] + failure_reg_probes['dot_successes'].fillna(0)
failure_reg_probes['do53_failure_rate'] = failure_reg_probes['do53_failures'] / failure_reg_probes['do53_total']
failure_reg_probes['dot_failure_rate'] = failure_reg_probes['dot_failures'] / failure_reg_probes['dot_total']

In [105]:
failure_reg_probes.reset_index().rename(columns={'index' : 'continent_code'})

,continent_code,do53_failures,dot_failures,do53_successes,dot_successes,do53_total,dot_total,do53_failure_rate,dot_failure_rate
0,AF,12678.0,80363.0,80319.0,NaN,92997.0,80363.0,0.136327,1.0
1,AS,35700.0,489222.0,424178.0,NaN,459878.0,489222.0,0.077629,1.0
2,EU,408814.0,4132011.0,2925638.0,NaN,3334452.0,4132011.0,0.122603,1.0
3,NA,87112.0,1169135.0,835197.0,NaN,922309.0,1169135.0,0.094450,1.0
4,OC,17979.0,202851.0,155377.0,NaN,173356.0,202851.0,0.103711,1.0
5,SA,11231.0,138434.0,114448.0,NaN,125679.0,138434.0,0.089363,1.0


In [106]:
with sqlite3.connect('../data/loc-res.db') as conn:
    failure_reg_probes.reset_index().rename(columns={'index' : 'continent_code'}
                                           ).to_sql('failure_rates_regular', index=False,
                                                    if_exists='replace', con=conn)

In [107]:
failure_special_probes = pd.DataFrame([do53_special_failures.groupby('continent_code').size(),
                                       dot_special_failures.groupby('continent_code').size(),
                                       do53_special.groupby('continent_code').size(),
                                       dot_special.groupby('continent_code').size()])

failure_special_probes = failure_special_probes.transpose()
failure_special_probes.columns = ['do53_failures', 'dot_failures', 'do53_successes', 'dot_successes']
failure_special_probes['do53_total'] = failure_special_probes['do53_failures'] + failure_special_probes['do53_successes']
failure_special_probes['dot_total'] = failure_special_probes['dot_failures'] + failure_special_probes['dot_successes'].fillna(0)
failure_special_probes['do53_failure_rate'] = failure_special_probes['do53_failures'] / failure_special_probes['do53_total']
failure_special_probes['dot_failure_rate'] = failure_special_probes['dot_failures'] / failure_special_probes['dot_total']

In [108]:
do53_special.groupby(['prb_id', 'country_code', 'asn', 'as_holder']).size().reset_index()

,prb_id,country_code,asn,as_holder,0
0,10342,US,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU",1365
1,14377,DE,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",2765
2,16233,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1392
3,17525,FR,60197.0,"LDN, FR",1399
4,17609,SE,1653.0,"SUNET SUNET Swedish University Network, EU",1399
5,18275,US,34939.0,"NEXTDNS, US",1400
6,20324,NL,3265.0,"XS4ALL-NL Amsterdam, NL",4197
7,22890,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1389
8,23214,SE,42303.0,"FIBERDIREKT, SE",2786
9,29989,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1398


### --> all special probes except for one (probe 30899) use only one local resolver

Note that probes can still have additional resolvers since this is only counting ones that respond successfully at least once. Probes could have additional resolvers but those fail for both DNS and DoT ALL THE TIME.

In [109]:
tmp_special_probes = true_local[true_local['prb_id'].isin(special_probes)]

In [110]:
tmp_special_probes_dot = tmp_special_probes[tmp_special_probes['is_tls'] == 1]

In [111]:
tmp_special_probes_dot

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1626059,22217033,google.com,None,10342,NaN,NaN,None,None,1562166010,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
1626060,22217033,google.com,None,10342,109.980,44.0,192.168.0.166,103.247.37.37,1562166010,1,300.0,NOERROR,0,None,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU"
1627863,22217033,google.com,None,14377,454.226,124.0,192.168.1.238,192.109.42.42,1562166007,1,300.0,NOERROR,0,None,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE"
1627864,22217033,google.com,None,14377,NaN,NaN,None,None,1562166007,1,NaN,None,1,{u'TUCONNECT': u'Connection refused'},NaN,NaN
1628357,22217033,google.com,None,16233,NaN,NaN,None,None,1562166003,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721845,22299007,sberbank.ru,None,30899,162.648,219.0,192.168.42.3,46.182.19.48,1562697214,1,3598.0,NOERROR,0,None,43847.0,"NBISERV-AS, DE"
23722390,22299007,sberbank.ru,None,32271,254.773,56.0,192.168.0.20,176.103.130.132,1562697214,1,2155.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722391,22299007,sberbank.ru,None,32271,226.664,56.0,192.168.0.20,176.103.130.134,1562697214,1,932.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722468,22299007,sberbank.ru,None,32430,NaN,NaN,None,None,1562697212,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [112]:
tmp_special_probes_dot[tmp_special_probes_dot['prb_id'] == 30899]

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1633086,22217033,google.com,None,30899,NaN,NaN,None,None,1562166007,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
1633087,22217033,google.com,None,30899,NaN,NaN,None,None,1562166007,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
1642143,22217061,youtube.com,None,30899,NaN,NaN,None,None,1562166072,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
1642144,22217061,youtube.com,None,30899,NaN,NaN,None,None,1562166072,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
1651189,22217067,facebook.com,None,30899,NaN,NaN,None,None,1562166136,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23704199,22298993,avito.ru,None,30899,NaN,NaN,None,None,1562697088,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
23713020,22298999,rambler.ru,None,30899,NaN,NaN,None,None,1562697150,1,NaN,None,1,{u'TUCONNECT': u'Connection refused'},NaN,NaN
23713021,22298999,rambler.ru,None,30899,163.736,310.0,192.168.42.3,46.182.19.48,1562697150,1,300.0,NOERROR,0,None,43847.0,"NBISERV-AS, DE"
23721844,22299007,sberbank.ru,None,30899,NaN,NaN,None,None,1562697214,1,NaN,None,1,{u'TUCONNECT': u'Connection refused'},NaN,NaN


In [113]:
tmp_special_probes_dot[tmp_special_probes_dot['prb_id'] == 30899].groupby('msm_id').size().reset_index()

,msm_id,0
0,22217033,2
1,22217061,2
2,22217067,2
3,22217079,2
4,22217089,2
...,...,...
1356,22298955,2
1357,22298986,2
1358,22298993,2
1359,22298999,2


In [114]:
_[_[0] != 2] 

,msm_id,0


In [115]:
# --> need to subtract 1361 from DoT failures and total (special probes) for EU (DE) probe
# (i.e., subtract len(Out[113]) in this case)

In [116]:
failure_special_probes.at['EU', 'dot_failures'] = failure_special_probes.at['EU', 'dot_failures'] - 1361
failure_special_probes.at['EU', 'dot_total'] = failure_special_probes.at['EU', 'dot_total'] - 1361

In [117]:
failure_special_probes.at['EU', 'dot_failure_rate'] = failure_special_probes.at['EU', 'dot_failures'] / failure_special_probes.at['EU', 'dot_total']

In [118]:
failure_special_probes.reset_index().rename(columns={'index' : 'continent_code'})

,continent_code,do53_failures,dot_failures,do53_successes,dot_successes,do53_total,dot_total,do53_failure_rate,dot_failure_rate
0,EU,1625,12343,26894,18312,28519,30655,0.056980,0.402642
1,NA,731,1394,3399,2790,4130,4184,0.176998,0.333174


In [119]:
with sqlite3.connect('../data/loc-res.db') as conn:
    failure_special_probes.reset_index().rename(columns={'index' : 'continent_code'}
                                               ).to_sql('failure_rates_special', index=False,
                                                        if_exists='replace', con=conn)

In [120]:
dot.groupby(['country_code', 'asn', 'as_holder'])['rt'].quantile(0.05).reset_index()

,country_code,asn,as_holder,rt
0,DE,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",191.90675
1,DE,43847.0,"NBISERV-AS, DE",161.37285
2,FR,60197.0,"LDN, FR",122.72300
3,NL,3265.0,"XS4ALL-NL Amsterdam, NL",147.42655
4,RO,199274.0,"MNGTNET, RU",223.23405
5,SE,1653.0,"SUNET SUNET Swedish University Network, EU",66.42740
6,SE,42303.0,"FIBERDIREKT, SE",117.80155
7,US,34939.0,"NEXTDNS, US",383.77400
8,US,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU",104.00170


In [121]:
dot.groupby(['prb_id', 'country_code', 'asn', 'as_holder'])['rt'].quantile(0.05).reset_index().sort_values('asn')

,prb_id,country_code,asn,as_holder,rt
4,17609,SE,1653.0,"SUNET SUNET Swedish University Network, EU",66.42740
2,16233,NL,3265.0,"XS4ALL-NL Amsterdam, NL",148.06850
6,20324,NL,3265.0,"XS4ALL-NL Amsterdam, NL",156.55865
7,22890,NL,3265.0,"XS4ALL-NL Amsterdam, NL",146.84850
9,29989,NL,3265.0,"XS4ALL-NL Amsterdam, NL",145.90230
12,32430,NL,3265.0,"XS4ALL-NL Amsterdam, NL",148.34625
1,14377,DE,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",191.90675
5,18275,US,34939.0,"NEXTDNS, US",383.77400
8,23214,SE,42303.0,"FIBERDIREKT, SE",117.80155
10,30899,DE,43847.0,"NBISERV-AS, DE",161.37285


In [122]:
dot.groupby(['prb_id', 'country_code', 'asn', 'as_holder']
           )['rt'].quantile(0.05).reset_index().sort_values('asn').reset_index(drop=True)

,prb_id,country_code,asn,as_holder,rt
0,17609,SE,1653.0,"SUNET SUNET Swedish University Network, EU",66.42740
1,16233,NL,3265.0,"XS4ALL-NL Amsterdam, NL",148.06850
2,20324,NL,3265.0,"XS4ALL-NL Amsterdam, NL",156.55865
3,22890,NL,3265.0,"XS4ALL-NL Amsterdam, NL",146.84850
4,29989,NL,3265.0,"XS4ALL-NL Amsterdam, NL",145.90230
5,32430,NL,3265.0,"XS4ALL-NL Amsterdam, NL",148.34625
6,14377,DE,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",191.90675
7,18275,US,34939.0,"NEXTDNS, US",383.77400
8,23214,SE,42303.0,"FIBERDIREKT, SE",117.80155
9,30899,DE,43847.0,"NBISERV-AS, DE",161.37285


In [123]:
_[['country_code', 'asn', 'as_holder', 'rt']]

,country_code,asn,as_holder,rt
0,SE,1653.0,"SUNET SUNET Swedish University Network, EU",66.42740
1,NL,3265.0,"XS4ALL-NL Amsterdam, NL",148.06850
2,NL,3265.0,"XS4ALL-NL Amsterdam, NL",156.55865
3,NL,3265.0,"XS4ALL-NL Amsterdam, NL",146.84850
4,NL,3265.0,"XS4ALL-NL Amsterdam, NL",145.90230
5,NL,3265.0,"XS4ALL-NL Amsterdam, NL",148.34625
6,DE,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",191.90675
7,US,34939.0,"NEXTDNS, US",383.77400
8,SE,42303.0,"FIBERDIREKT, SE",117.80155
9,DE,43847.0,"NBISERV-AS, DE",161.37285


In [124]:
_.round(1).to_latex()

'\\begin{tabular}{llrlr}\n\\toprule\n{} & country\\_code &       asn &                                        as\\_holder &     rt \\\\\n\\midrule\n0  &           SE &    1653.0 &       SUNET SUNET Swedish University Network, EU &   66.4 \\\\\n1  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  148.1 \\\\\n2  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  156.6 \\\\\n3  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  146.8 \\\\\n4  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  145.9 \\\\\n5  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  148.3 \\\\\n6  &           DE &   29670.0 &  IN-BERLIN-AS Individual Network Berlin e.V., DE &  191.9 \\\\\n7  &           US &   34939.0 &                                      NEXTDNS, US &  383.8 \\\\\n8  &           SE &   42303.0 &                                  FIBERDIREKT, SE &  

In [125]:
print(_)

\begin{tabular}{llrlr}
\toprule
{} & country\_code &       asn &                                        as\_holder &     rt \\
\midrule
0  &           SE &    1653.0 &       SUNET SUNET Swedish University Network, EU &   66.4 \\
1  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  148.1 \\
2  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  156.6 \\
3  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  146.8 \\
4  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  145.9 \\
5  &           NL &    3265.0 &                          XS4ALL-NL Amsterdam, NL &  148.3 \\
6  &           DE &   29670.0 &  IN-BERLIN-AS Individual Network Berlin e.V., DE &  191.9 \\
7  &           US &   34939.0 &                                      NEXTDNS, US &  383.8 \\
8  &           SE &   42303.0 &                                  FIBERDIREKT, SE &  117.8 \\
9  &           DE &   43847

In [126]:
dot.groupby(['prb_id', 'country_code', 'asn', 'as_holder']
           )['rt'].describe(percentiles=[0.05,0.25,0.50,0.75,0.95]).reset_index().sort_values('asn').reset_index(drop=True)

,prb_id,country_code,asn,as_holder,count,mean,std,min,5%,25%,50%,75%,95%,max
0,17609,SE,1653.0,"SUNET SUNET Swedish University Network, EU",1392.0,68.801388,9.978848,65.224,66.42740,67.27250,67.8375,68.44200,70.48145,339.317
1,16233,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1395.0,289.077553,338.439141,146.218,148.06850,149.42350,151.5850,247.11950,1005.39340,3451.883
2,20324,NL,3265.0,"XS4ALL-NL Amsterdam, NL",2792.0,171.520614,109.424318,153.819,156.55865,158.16250,159.8100,161.34950,168.49345,3213.378
3,22890,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1394.0,428.401622,488.408945,145.189,146.84850,152.71625,201.6465,456.16075,1677.20185,4012.359
4,29989,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1395.0,391.122885,245.001205,143.580,145.90230,152.08450,403.6220,530.94450,666.59600,2470.312
5,32430,NL,3265.0,"XS4ALL-NL Amsterdam, NL",1396.0,316.230156,396.215762,146.962,148.34625,149.68775,152.0475,279.99275,1043.14900,3344.099
6,14377,DE,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE",1396.0,495.311597,416.800667,174.964,191.90675,228.42625,353.6550,581.77575,1300.43475,3426.581
7,18275,US,34939.0,"NEXTDNS, US",1396.0,400.994604,67.205002,377.606,383.77400,386.65950,388.3810,391.08450,444.91950,1533.130
8,23214,SE,42303.0,"FIBERDIREKT, SE",2790.0,375.950487,335.749498,104.557,117.80155,155.03475,245.4330,468.65000,1039.88250,2279.510
9,30899,DE,43847.0,"NBISERV-AS, DE",194.0,476.325510,650.319331,160.651,161.37285,162.41375,164.4795,497.40150,1802.32745,3634.205


In [127]:
_[['country_code', 'asn', '5%', '25%', '50%', '75%', '95%']]

,country_code,asn,5%,25%,50%,75%,95%
0,SE,1653.0,66.42740,67.27250,67.8375,68.44200,70.48145
1,NL,3265.0,148.06850,149.42350,151.5850,247.11950,1005.39340
2,NL,3265.0,156.55865,158.16250,159.8100,161.34950,168.49345
3,NL,3265.0,146.84850,152.71625,201.6465,456.16075,1677.20185
4,NL,3265.0,145.90230,152.08450,403.6220,530.94450,666.59600
5,NL,3265.0,148.34625,149.68775,152.0475,279.99275,1043.14900
6,DE,29670.0,191.90675,228.42625,353.6550,581.77575,1300.43475
7,US,34939.0,383.77400,386.65950,388.3810,391.08450,444.91950
8,SE,42303.0,117.80155,155.03475,245.4330,468.65000,1039.88250
9,DE,43847.0,161.37285,162.41375,164.4795,497.40150,1802.32745


In [128]:
print(_.round(1).to_latex())

\begin{tabular}{llrrrrrr}
\toprule
{} & country\_code &       asn &     5\% &    25\% &    50\% &    75\% &     95\% \\
\midrule
0  &           SE &    1653.0 &   66.4 &   67.3 &   67.8 &   68.4 &    70.5 \\
1  &           NL &    3265.0 &  148.1 &  149.4 &  151.6 &  247.1 &  1005.4 \\
2  &           NL &    3265.0 &  156.6 &  158.2 &  159.8 &  161.3 &   168.5 \\
3  &           NL &    3265.0 &  146.8 &  152.7 &  201.6 &  456.2 &  1677.2 \\
4  &           NL &    3265.0 &  145.9 &  152.1 &  403.6 &  530.9 &   666.6 \\
5  &           NL &    3265.0 &  148.3 &  149.7 &  152.0 &  280.0 &  1043.1 \\
6  &           DE &   29670.0 &  191.9 &  228.4 &  353.7 &  581.8 &  1300.4 \\
7  &           US &   34939.0 &  383.8 &  386.7 &  388.4 &  391.1 &   444.9 \\
8  &           SE &   42303.0 &  117.8 &  155.0 &  245.4 &  468.6 &  1039.9 \\
9  &           DE &   43847.0 &  161.4 &  162.4 &  164.5 &  497.4 &  1802.3 \\
10 &           FR &   60197.0 &  122.7 &  128.3 &  155.9 &  275.0 &   654.2 \\
11

In [129]:
true_local[true_local['prb_id'].isin(special_probes)]

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
115,22202814,google.com,None,10342,1.173,44.0,192.168.0.166,192.168.0.11,1562122810,0,243.0,NOERROR,0,None,NaN,NaN
116,22202814,google.com,None,10342,10.808,44.0,192.168.0.166,103.247.37.37,1562122810,0,189.0,NOERROR,0,None,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU"
1717,22202814,google.com,None,14377,21.364,124.0,192.168.1.238,192.109.42.42,1562122807,0,205.0,NOERROR,0,None,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE"
1718,22202814,google.com,None,14377,103.558,318.0,192.168.1.238,192.109.42.41,1562122807,0,300.0,NOERROR,0,None,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE"
2165,22202814,google.com,None,16233,4.466,44.0,80.127.100.88,194.109.6.66,1562122804,0,193.0,NOERROR,0,None,3265.0,"XS4ALL-NL Amsterdam, NL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721845,22299007,sberbank.ru,None,30899,162.648,219.0,192.168.42.3,46.182.19.48,1562697214,1,3598.0,NOERROR,0,None,43847.0,"NBISERV-AS, DE"
23722390,22299007,sberbank.ru,None,32271,254.773,56.0,192.168.0.20,176.103.130.132,1562697214,1,2155.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722391,22299007,sberbank.ru,None,32271,226.664,56.0,192.168.0.20,176.103.130.134,1562697214,1,932.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722468,22299007,sberbank.ru,None,32430,NaN,NaN,None,None,1562697212,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [130]:
_[_['is_tls'] == 1]

,msm_id,domain,resolver,prb_id,rt,size,src_addr,dst_addr,timestamp,is_tls,ttl,return_code,err,err_msg,asn,as_holder
1626059,22217033,google.com,None,10342,NaN,NaN,None,None,1562166010,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN
1626060,22217033,google.com,None,10342,109.980,44.0,192.168.0.166,103.247.37.37,1562166010,1,300.0,NOERROR,0,None,64089.0,"DNSFILTER-AS-AP DNSFilter, Inc., AU"
1627863,22217033,google.com,None,14377,454.226,124.0,192.168.1.238,192.109.42.42,1562166007,1,300.0,NOERROR,0,None,29670.0,"IN-BERLIN-AS Individual Network Berlin e.V., DE"
1627864,22217033,google.com,None,14377,NaN,NaN,None,None,1562166007,1,NaN,None,1,{u'TUCONNECT': u'Connection refused'},NaN,NaN
1628357,22217033,google.com,None,16233,NaN,NaN,None,None,1562166003,1,NaN,None,1,{u'timeout': 5000},NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721845,22299007,sberbank.ru,None,30899,162.648,219.0,192.168.42.3,46.182.19.48,1562697214,1,3598.0,NOERROR,0,None,43847.0,"NBISERV-AS, DE"
23722390,22299007,sberbank.ru,None,32271,254.773,56.0,192.168.0.20,176.103.130.132,1562697214,1,2155.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722391,22299007,sberbank.ru,None,32271,226.664,56.0,192.168.0.20,176.103.130.134,1562697214,1,932.0,NOERROR,0,None,199274.0,"MNGTNET, RU"
23722468,22299007,sberbank.ru,None,32430,NaN,NaN,None,None,1562697212,1,NaN,None,1,{u'TU_BAD_ADDR': True},NaN,NaN


In [131]:
tmp_local = _

In [132]:
tmp_local.groupby(['prb_id']).size().reset_index()

,prb_id,0
0,10342,2788
1,14377,2792
2,16233,1397
3,17525,4188
4,17609,1392
5,18275,1396
6,20324,4188
7,22890,1396
8,23214,4185
9,29989,4188


In [133]:
tmp_special_total_dot = pd.DataFrame(tmp_local.groupby(['prb_id']).size())

In [134]:
# manual subtraction for probe 30899, 1361 results
tmp_special_total_dot.at[30899, 0] = tmp_special_total_dot.at[30899, 0] - 1361

In [135]:
tmp_special_total_dot = tmp_special_total_dot.reset_index().rename(columns={0 : 'total'})

In [136]:
tmp_special_total_dot

,prb_id,total
0,10342,2788
1,14377,2792
2,16233,1397
3,17525,4188
4,17609,1392
5,18275,1396
6,20324,4188
7,22890,1396
8,23214,4185
9,29989,4188
